<a href="https://colab.research.google.com/github/aghadavood/NLP/blob/main/fine_tuning_PersianData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'appstreamlit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5219829%2F8702914%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240618%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240618T215720Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D72171ec7a7f179af7bb09b0ba583a5c4f3636f1cec90bbc8ff337bfc81671c9723885d5075973c41b22b2f1d3f7c24835fa42c0e7feaad4a3472a6f56a917e6e307c3ae30cb595049e2f03ee2cde1b6a5476c86c462a8d8ac7519befc33c32f471708ba31c4d698f78f2b6c938dfefcf7a6e171a467980f7797e070219e7543843fb2f8351f05307f10a1bcaf127377e3af829fbf749575fae1e4d3cef29a09c6da1c134ab6d818f20127d688de187327eb51475a3417e8f5f9afe7498f1df29bf4e80ab3a9ec957031cf1cb31ab80b20890a9a5bb78a34db5e94b12374ea051f917e205746b5d32ea5bfc93041c1aae7c98ca8c86aa6a38bc1362bf2961bb0a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# Install necessary packages
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install tokenizers

# Import necessary libraries
from datasets import load_dataset, DatasetDict, Dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator, TrainerCallback

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info/METADATA'



2024-06-10 15:15:56.847122: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 15:15:56.847226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 15:15:56.958495: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Load and prepare dataset
def load_and_prepare_dataset(dataset_path):
    dataset = load_dataset(dataset_path)  # Replace with your dataset path

    # Ensure the dataset has the required columns
    required_columns = ['question', 'context', 'answers']
    for split in dataset:
        for column in required_columns:
            if column not in dataset[split].column_names:
                raise ValueError(f"Missing required column '{column}' in {split} split")

    return dataset

dataset_path = "SajjadAyoubi/persian_qa"  # Update with your dataset path
dataset = load_and_prepare_dataset(dataset_path)
print(dataset)

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})


In [ ]:
# Tokenize dataset
def tokenize_dataset(dataset, tokenizer):
    def preprocess_function(examples):
        questions = examples['question']
        contexts = examples['context']
        answers = examples['answers']

        inputs = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512)

        start_positions = []
        end_positions = []

        for i in range(len(questions)):
            context = contexts[i]
            if len(answers[i]['text']) == 0 or len(answers[i]['answer_start']) == 0:
                # Handle missing or empty answer cases
                start_positions.append(0)
                end_positions.append(0)
                continue

            answer = answers[i]['text'][0]
            answer_start = answers[i]['answer_start'][0]
            answer_end = answer_start + len(answer)

            # Tokenize context with the fast tokenizer
            tokenized_context = tokenizer(context, truncation=True, padding='max_length', max_length=512)

            # Find start and end token positions
            start_token_idx = tokenized_context.char_to_token(answer_start)
            end_token_idx = tokenized_context.char_to_token(answer_end - 1)

            # Handle edge case where char_to_token returns None
            if start_token_idx is None:
                start_token_idx = 0
            if end_token_idx is None:
                end_token_idx = len(tokenized_context.input_ids) - 1

            start_positions.append(start_token_idx)
            end_positions.append(end_token_idx)

        inputs.update({'start_positions': start_positions, 'end_positions': end_positions})
        return inputs

    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    return tokenized_datasets
# Initialize the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('HooshvareLab/bert-fa-base-uncased')
tokenized_datasets = tokenize_dataset(dataset, tokenizer)
print(tokenized_datasets)


vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 930
    })
})


In [ ]:
from transformers import TrainerCallback, TrainingArguments, Trainer, DefaultDataCollator

# Initialize data collator
data_collator = DefaultDataCollator()

# Define early stopping callback
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3):
        self.early_stopping_patience = early_stopping_patience
        self.best_score = None
        self.counter = 0

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        score = metrics.get("eval_loss")
        if self.best_score is None or score < self.best_score:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True

# Define training arguments with early stopping
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,  # Set to a higher number and rely on early stopping
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Use eval_loss to determine the best model
    greater_is_better=False,  # Since lower eval_loss is better
)

# Initialize the model
def initialize_model(model_name='HooshvareLab/bert-fa-base-uncased'):
    model = BertForQuestionAnswering.from_pretrained(model_name)
    return model

model = initialize_model()



# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the fine-tuned model function
def save_model(trainer, output_dir='fine_tuned_model'):
    trainer.save_model(output_dir)

# Save the model
save_model(trainer)

# List the contents of the current working directory
print("Contents of the current working directory:", os.listdir('.'))

# List the contents of the 'fine_tuned_model' directory
print("Contents of the fine_tuned_model directory:", os.listdir('fine_tuned_model'))



/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


KeyboardInterrupt: 

In [ ]:
import shutil
import os

# Compress the fine_tuned_model directory into a zip file
output_dir = 'fine_tuned_model'
shutil.make_archive(output_dir, 'zip', output_dir)

# Verify the zip file is created
print(f"Contents of the current working directory: {os.listdir('.')}")

# The zip file 'fine_tuned_model.zip' should now be available for download in the Kaggle output section.


Contents of the current working directory: ['results', 'fine_tuned_model', '.virtual_documents', 'wandb', 'fine_tuned_model.zip']


the result are :
Evaluation results: {'eval_loss': 2.898200273513794, 'eval_runtime': 16.3971, 'eval_samples_per_second': 56.717, 'eval_steps_per_second': 3.598, 'epoch': 5.0}

The model shows a decreasing trend in training loss, which is good.
The validation loss decreases up to a certain point but then starts increasing, suggesting overfitting.

**better result so i have some solutions:
**
1. Increase Regularization
2. Reduce learning rate
3. data augmentation

we should try one solution and see the result and try another solution.
so we can understand the impact of each change.



In [ ]:
#Adjust Learning Rate:

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-5,  # Reduced learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=DefaultDataCollator(),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.677900,3.332214
2,3.162200,3.062102
3,2.817300,2.992796
4,2.229800,3.024501
5,2.123200,3.160560


Evaluation results: {'eval_loss': 2.9927964210510254, 'eval_runtime': 16.3812, 'eval_samples_per_second': 56.772, 'eval_steps_per_second': 3.602, 'epoch': 5.0}


now the results show that , in training loss we have a decreese for both approaches(adjusting ealrly stopping,adjusting learning rate). but in validation loss, in learning rate shows a slight improvment in validation loss, particluarly in epoch 3.
the validation loss and it reduce more than other approach.
now Hyperparameter Tuning with Grid Search:

In [ ]:
from transformers import Trainer, TrainingArguments, DefaultDataCollator, TrainerCallback
from sklearn.model_selection import ParameterGrid


# Initialize data collator
data_collator = DefaultDataCollator()

# Define early stopping callback
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3):
        self.early_stopping_patience = early_stopping_patience
        self.best_score = None
        self.counter = 0

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        score = metrics.get("eval_loss")
        if self.best_score is None or score < self.best_score:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True

# Define the grid of hyperparameters
param_grid = {
    'learning_rate': [2e-5, 3e-5, 5e-5],
    'per_device_train_batch_size': [16, 32],
    'num_train_epochs': [5, 10],
    'weight_decay': [0.01, 0.1],
}

# Initialize the model
def initialize_model(model_name='HooshvareLab/bert-fa-base-uncased'):
    model = BertForQuestionAnswering.from_pretrained(model_name)
    return model

model = initialize_model()

# Iterate over all combinations of hyperparameters
for params in ParameterGrid(param_grid):
    print(f"Training with params: {params}")

    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=params['learning_rate'],
        per_device_train_batch_size=params['per_device_train_batch_size'],
        per_device_eval_batch_size=16,
        num_train_epochs=params['num_train_epochs'],
        weight_decay=params['weight_decay'],
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        data_collator=DefaultDataCollator(),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    eval_results = trainer.evaluate()
    print(f"Evaluation results: {eval_results}")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training with params: {'learning_rate': 2e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 16, 'weight_decay': 0.01}


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Epoch,Training Loss,Validation Loss
1,2.062100,3.227532
2,1.657000,3.542318
3,1.574600,3.665837
4,1.051300,4.113561


Evaluation results: {'eval_loss': 3.227532386779785, 'eval_runtime': 16.1383, 'eval_samples_per_second': 57.627, 'eval_steps_per_second': 3.656, 'epoch': 4.0}
Training with params: {'learning_rate': 2e-05, 'num_train_epochs': 5, 'per_device_train_batch_size': 32, 'weight_decay': 0.01}


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 106.12 MiB is free. Process 2344 has 15.79 GiB memory in use. Of the allocated memory 15.29 GiB is allocated by PyTorch, and 209.28 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# set params based the best of grid searc
from transformers import TrainerCallback, TrainingArguments, Trainer, DefaultDataCollator

# Initialize data collator
data_collator = DefaultDataCollator()

# Define early stopping callback
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, early_stopping_patience=3):
        self.early_stopping_patience = early_stopping_patience
        self.best_score = None
        self.counter = 0

    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        score = metrics.get("eval_loss")
        if self.best_score is None or score < self.best_score:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.early_stopping_patience:
                control.should_training_stop = True

# Define training arguments with early stopping
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Set to a higher number and rely on early stopping
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Use eval_loss to determine the best model
    greater_is_better=False,  # Since lower eval_loss is better
)

# Initialize the model
def initialize_model(model_name='HooshvareLab/bert-fa-base-uncased'):
    model = BertForQuestionAnswering.from_pretrained(model_name)
    return model

model = initialize_model()



# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the fine-tuned model function
def save_model(trainer, output_dir='fine_tuned_model'):
    trainer.save_model(output_dir)

# Save the model
save_model(trainer)



